## 3. Tools

<div style="text-align: right"> Initial issue : 2025.11.05 </div>
<div style="text-align: right"> last update : 2025.11.05 </div>

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### 도구 생성
기본 도구 정의: @tool 데코레이터를 사용. 이때 함수의 docstring 이 모델이 도구를 언제 사용할지 이해하는데 도움이됨

In [2]:
from langchain.tools import tool

In [3]:
@tool
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"

In [4]:
# 도구 정보 확인 가능
print(search_database)
print(f"Tool name: {search_database.name}")
print(f"Tool description: {search_database.description}")

name='search_database' description='Search the customer database for records matching the query.\n\n    Args:\n        query: Search terms to look for\n        limit: Maximum number of results to return' args_schema=<class 'langchain_core.utils.pydantic.search_database'> func=<function search_database at 0x10aa477e0>
Tool name: search_database
Tool description: Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return


### 도구 속성 커스터 마이징
커스텀 도구 이름: 기본적으로 함수 이름에서 가져오는데, 더 설명이 필요한 경우 재정의 가능

In [5]:
@tool("web_search")  # 커스텀 이름
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"

In [6]:
print(search)
print(f"Tool name: {search.name}")
print(f"Tool description: {search.description}")

name='web_search' description='Search the web for information.' args_schema=<class 'langchain_core.utils.pydantic.web_search'> func=<function search at 0x1101deac0>
Tool name: web_search
Tool description: Search the web for information.


커스텀 도구 설명: 비슷하게 모델 가이들을 위해 자동 생성된 도구 설명 재정의 가능

In [7]:
@tool("calculator", description="Performs arithmetic calculations. Use this for any math problems.")
def calc(expression: str) -> str:
    """Evaluate mathematical expressions."""
    return str(eval(expression))

In [8]:
print(f"Tool description: {calc.description}")

Tool description: Performs arithmetic calculations. Use this for any math problems.


### 고급 스키마 정의
pydantic, json으로 복잡한 입력 정의 가능

pydantic

In [9]:
from pydantic import BaseModel, Field
from typing import Literal

class WeatherInput(BaseModel):
    """Input for weather queries."""
    location: str = Field(description="City name or coordinates")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit preference"
    )
    include_forecast: bool = Field(
        default=False,
        description="Include 5-day forecast"
    )

In [10]:
@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

In [11]:
print(get_weather)

name='get_weather' description='Get current weather and optional forecast.' args_schema=<class '__main__.WeatherInput'> func=<function get_weather at 0x1101df1a0>


In [12]:
# 도구 테스트
print(get_weather.invoke({"location": "Seoul", "units": "celsius", "include_forecast": True}))

Current weather in Seoul: 22 degrees C
Next 5 days: Sunny


json 스키마

In [13]:
weather_schema = {
    "type": "object",
    "properties": {
        "location": {"type": "string"},
        "units": {"type": "string"},
        "include_forecast": {"type": "boolean"}
    },
    "required": ["location", "units", "include_forecast"]
}

@tool(args_schema=weather_schema)
def get_weather_v2(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

In [14]:
# 도구 테스트
print(get_weather_v2.invoke({"location": "Seoul", "units": "celsius", "include_forecast": True}))

Current weather in Seoul: 22 degrees C
Next 5 days: Sunny


### 컨텍스트 접근
도구는 에이전트 상태, 런타임 컨텍스트 및 장기 메모리에 액세스할 수 있을 때 가장 강력합니다.  
이를 통해 도구는 컨텍스트 인식 결정을 내리고, 응답을 개인화하며, 대화 전반에 걸쳐 정보를 유지할 수 있습니다

도구는 `ToolRuntime` 매개변수를 통해 런타임 정보에 액세스할 수 있으며, 다음을 제공합니다:

- **State** - 실행을 통해 흐르는 변경 가능한 데이터 (메시지, 카운터, 커스텀 필드)
- **Context** - 사용자 ID, 세션 세부 정보 또는 애플리케이션별 구성과 같은 불변 구성
- **Store** - 대화 전반에 걸친 영구 장기 메모리
- **Stream Writer** - 도구가 실행될 때 커스텀 업데이트 스트리밍
- **Config** - 실행을 위한 RunnableConfig
- **Tool Call ID** - 현재 도구 호출의 ID

### ToolRuntime
ToolRuntime을 사용하면 단일 매개변수로 모든 런타임 정보에 액세스 가능   
도구 시그니처에 runtime: ToolRuntime 추가하면, LLM에 노출되지 않고 자동 주입

상태 접근: toolruntime을 사용하여 현재 그래프 상태에 접근 가능

In [15]:
from langchain.tools import tool, ToolRuntime

# 현재 대화 상태 접근
@tool
def summarize_conversation(
    runtime: ToolRuntime
) -> str:
    """Summarize the conversation so far."""
    messages = runtime.state["messages"]

    human_msgs = sum(1 for m in messages if m.__class__.__name__ == "HumanMessage")
    ai_msgs = sum(1 for m in messages if m.__class__.__name__ == "AIMessage")
    tool_msgs = sum(1 for m in messages if m.__class__.__name__ == "ToolMessage")

    return f"Conversation has {human_msgs} user messages, {ai_msgs} AI responses, and {tool_msgs} tool results"

# 커스텀 상태 필드 접근
@tool
def get_user_preference(
    pref_name: str,
    runtime: ToolRuntime  # ToolRuntime 매개변수는 모델에 보이지 않습니다
) -> str:
    """Get a user preference value."""
    preferences = runtime.state.get("user_preferences", {})
    return preferences.get(pref_name, "Not set")

상태 업데이트: Command를 사용하여 에이전트 상태를 업데이트하고, 그래프 실행 제어 가능

In [16]:
from langgraph.types import Command
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.tools import tool, ToolRuntime

In [17]:
# 모든 메시지를 제거하여 대화 기록 업데이트
@tool
def clear_conversation() -> Command:
    """Clear the conversation history."""
    return Command(
        update={
            "messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES)],
        }
    )

# 에이전트 상태에서 user_name 업데이트
@tool
def update_user_name(
    new_name: str,
    runtime: ToolRuntime
) -> Command:
    """Update the user's name."""
    return Command(update={"user_name": new_name})

컨텍스트: runtime.context를 통해 사용자 ID, 세션 세부 정보 또는 어플리케이션별 구성과 같은 불변 구성 및 컨텍스트 데이터 엑세스 가능

In [18]:
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime

# 사용자 데이터베이스 시뮬레이션
USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com"
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com"
    }
}

@dataclass
class UserContext:
    user_id: str

@tool
def get_account_info(runtime: ToolRuntime[UserContext]) -> str:
    """Get the current user's account information."""
    user_id = runtime.context.user_id

    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
    return "User not found"

In [19]:
# 에이전트 생성
model = ChatOpenAI(model="gpt-4.1-mini")
agent = create_agent(
    model,
    tools=[get_account_info],
    context_schema=UserContext,
    system_prompt="You are a financial assistant."
)

# 컨텍스트와 함께 에이전트 실행
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my current balance?"}]},
    context=UserContext(user_id="user123")
)

print(result["messages"][-1].content)

Your current balance is $5000.


### 메모리 (Store)
스토어를 사용하여 대화 전반에 걸쳐 영구 데이터에 액세스할 수 있습니다.   
스토어는 `runtime.store`를 통해 액세스되며 사용자별 또는 애플리케이션별 데이터를 저장하고 검색할 수 있습니다.

In [20]:
from typing import Any
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime

# 메모리 접근
@tool
def get_user_info(user_id: str, runtime: ToolRuntime) -> str:
    """Look up user info."""
    store = runtime.store
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"

# 메모리 업데이트
@tool
def save_user_info(user_id: str, user_info: dict[str, Any], runtime: ToolRuntime) -> str:
    """Save user info."""
    store = runtime.store
    store.put(("users",), user_id, user_info)
    return "Successfully saved user info."

In [21]:
# 스토어와 에이전트 생성
store = InMemoryStore()
model = ChatOpenAI(model="gpt-4.1-mini")
agent = create_agent(
    model,
    tools=[get_user_info, save_user_info],
    store=store
)

In [22]:
# 첫 번째 세션: 사용자 정보 저장
print("=== Saving user info ===")
result1 = agent.invoke({
    "messages": [{"role": "user", "content": "Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev"}]
})
print(result1["messages"][-1].content)

# 두 번째 세션: 사용자 정보 가져오기
print("\n=== Getting user info ===")
result2 = agent.invoke({
    "messages": [{"role": "user", "content": "Get user info for user with id 'abc123'"}]
})
print(result2["messages"][-1].content)

=== Saving user info ===
The user information for userid "abc123" with name "Foo", age 25, and email "foo@langchain.dev" has been saved successfully.

=== Getting user info ===
The user with ID 'abc123' is named Foo. They are 25 years old and their email address is foo@langchain.dev. Is there anything else you would like to know or do regarding this user?


### Stream Writer
`runtime.stream_writer`를 사용하여 도구가 실행될 때 커스텀 업데이트를 스트리밍할 수 있습니다.   
이는 도구가 수행하는 작업에 대해 사용자에게 실시간 피드백을 제공하는 데 유용합니다.

In [23]:
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_with_updates(city: str, runtime: ToolRuntime) -> str:
    """Get weather for a given city."""
    writer = runtime.stream_writer

    # 도구가 실행될 때 커스텀 업데이트 스트리밍
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")

    return f"It's always sunny in {city}!"

# 참고: runtime.stream_writer를 도구 내에서 사용하는 경우,
# 도구는 LangGraph 실행 컨텍스트 내에서 호출되어야 합니다.

### 종합 예제

지금까지 개념 모두 활용

In [24]:
from dataclasses import dataclass
from typing import Any
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langgraph.store.memory import InMemoryStore
from pydantic import BaseModel, Field

In [25]:
# 컨텍스트 스키마 정의
@dataclass
class AppContext:
    user_id: str
    session_id: str

# Pydantic 스키마를 사용한 도구
class NoteInput(BaseModel):
    """Input for saving a note."""
    title: str = Field(description="Title of the note")
    content: str = Field(description="Content of the note")

@tool(args_schema=NoteInput)
def save_note(title: str, content: str, runtime: ToolRuntime[AppContext]) -> str:
    """Save a note for the current user."""
    user_id = runtime.context.user_id
    store = runtime.store
    
    # 스토어에 노트 저장
    note_data = {"title": title, "content": content}
    store.put(("notes", user_id), title, note_data)
    
    return f"Successfully saved note: {title}"

@tool
def list_notes(runtime: ToolRuntime[AppContext]) -> str:
    """List all notes for the current user."""
    user_id = runtime.context.user_id
    store = runtime.store
    
    # 모든 노트 가져오기
    notes = store.search(("notes", user_id))
    
    if not notes:
        return "No notes found."
    
    result = "Your notes:\n"
    for note in notes:
        result += f"- {note.value['title']}: {note.value['content']}\n"
    
    return result

In [26]:
# 에이전트 설정
store = InMemoryStore()
model = ChatOpenAI(model="gpt-4.1-mini")

agent = create_agent(
    model,
    tools=[save_note, list_notes],
    context_schema=AppContext,
    store=store,
    system_prompt="You are a helpful note-taking assistant."
)

# 컨텍스트 정의
context = AppContext(user_id="user_001", session_id="session_123")

In [ ]:
# 노트 저장
print("=== Saving notes ===")
# result1 = agent.invoke( # 수정 필요
#     {"messages": [{"role": "user", "content": "Save a note titled 'Meeting' with content 'Team meeting at 3pm'"}]},
#     context=context
# )
# print(result1["messages"][-1].content)

# 노트 목록 조회
print("\n=== Listing notes ===")
result2 = agent.invoke(
    {"messages": [{"role": "user", "content": "Show me all my notes"}]},
    context=context
)
print(result2["messages"][-1].content)

=== Saving notes ===

=== Listing notes ===
You currently have no notes saved. Would you like to create a new note?
